In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
from enum import Enum
import time
import json
import pandas as pd

class WebDriver(Enum):
    Chrome = 1
    Edge = 2
    IE = 3

class OrangeHRMWebDriver:
    def __init__(self, webDriver):
        if webDriver == WebDriver.Chrome:
            self.driver = webdriver.Chrome(os.getenv("ChromeWebDriver"))
        elif webDriver == WebDriver.Edge:
            self.driver = webdriver.Edge(os.getenv("EdgeWebDriver"))
        else:
            self.driver = webdriver.Ie(os.getenv("IEWebDriver"))
        self.driver.get("http://localhost")
        self.TIMEOUT = 3
        with open("XPATH.json", "r") as f:
            self.xpath = json.load(f)
        self.addUserTestCases = pd.read_csv("Testcases/AddUserTestCases.csv", encoding="utf-8").to_dict(orient='records')
        self.addEmployeeTestCases = pd.read_csv("Testcases/AddEmployeeTestCases.csv", encoding="utf-8").to_dict(orient='records')

    def AdminLogin(self):
        username = WebDriverWait(self.driver, self.TIMEOUT).until(
            EC.presence_of_element_located((By.NAME, "username"))
        )
        password = WebDriverWait(self.driver, self.TIMEOUT).until(
            EC.presence_of_element_located((By.NAME, "password"))
        )
        username.send_keys("admin")
        password.send_keys("Admin@123456")
        loginButton = WebDriverWait(self.driver, self.TIMEOUT).until(
            EC.presence_of_element_located(
                (By.XPATH, "//button[text()=' Login ']"))
        )
        loginButton.click()

    def AdminAddUser(self, testcase):
        time.sleep(self.TIMEOUT)
        self.driver.get(self.xpath["AdminAddUserPage"]['Path'])
        saveButton = WebDriverWait(self.driver, self.TIMEOUT).until(
            EC.presence_of_element_located(
                (By.XPATH, self.xpath["AdminAddUserPage"]['SaveButton']))
        )

        if not pd.isna(testcase['UserRole']):
            self.driver.find_element(By.XPATH, self.xpath["AdminAddUserPage"]['UserRoleArrow']).click()
            self.driver.find_element(By.XPATH, self.xpath["AdminAddUserPage"]['UserRole' + testcase['UserRole']]).click()
        if not pd.isna(testcase['Status']):
            self.driver.find_element(By.XPATH, self.xpath["AdminAddUserPage"]['StatusArrow']).click()
            self.driver.find_element(By.XPATH, self.xpath["AdminAddUserPage"]['Status' + testcase['Status']]).click() 
        if not pd.isna(testcase['EmployeeName']):
            self.driver.find_element(By.XPATH, self.xpath["AdminAddUserPage"]['EmployeeName']).send_keys(testcase["EmployeeName"]) 
            time.sleep(self.TIMEOUT)
            WebDriverWait(self.driver, self.TIMEOUT).until(
                EC.presence_of_element_located(
                    (By.XPATH, self.xpath["AdminAddUserPage"]['EmployeeNameSearch']))
            ).click()
        self.driver.find_element(By.XPATH, self.xpath["AdminAddUserPage"]['Username']).send_keys(testcase["Username"]) if not pd.isna(testcase['Username']) else None
        self.driver.find_element(By.XPATH, self.xpath["AdminAddUserPage"]['Password']).send_keys(testcase["Password"]) if not pd.isna(testcase['Password']) else None
        self.driver.find_element(By.XPATH, self.xpath["AdminAddUserPage"]['ConfirmPassword']).send_keys(testcase["ConfirmPassword"]) if not pd.isna(testcase['ConfirmPassword']) else None
        
        time.sleep(self.TIMEOUT)
        saveButton.click()

        if testcase['Field'] == 'All':
            try:
                WebDriverWait(self.driver, self.TIMEOUT).until(
                    EC.presence_of_element_located(
                        (By.XPATH, self.xpath["AdminAddUserPage"]['SuccessNotification']))
                )
            except:
                print(f'Test case {testcase["No."]}: {"Success" if not testcase["ExpectedResult"] else "Fail"}')
                return
            print(f'Test case {testcase["No."]}: {"Success" if testcase["ExpectedResult"] else "Fail"}')
        else:
            try:
                WebDriverWait(self.driver, self.TIMEOUT).until(
                    EC.presence_of_element_located(
                        (By.XPATH, self.xpath["AdminAddUserPage"]['Invalid' + testcase['Field']]))
                )
            except:
                print(f'Test case {testcase["No."]}: {"Success" if testcase["ExpectedResult"] else "Fail"}')
                return
            print(f'Test case {testcase["No."]}: {"Success" if not testcase["ExpectedResult"] else "Fail"}')
    def AdminAddEmployee(self, testcase):
        time.sleep(self.TIMEOUT)
        self.driver.get(self.xpath["AdminAddEmployeePage"]['Path'])
        saveButton = WebDriverWait(self.driver, self.TIMEOUT).until(
            EC.presence_of_element_located(
                (By.XPATH, self.xpath["AdminAddEmployeePage"]['SaveButton']))
        )

        self.driver.find_element(By.NAME, "firstName").send_keys(testcase['Firstname']) if not pd.isna(testcase['Firstname']) else None
        self.driver.find_element(By.NAME, "middleName").send_keys(testcase['Middlename']) if not pd.isna(testcase['Middlename']) else None
        self.driver.find_element(By.NAME, "lastName").send_keys(testcase['Lastname']) if not pd.isna(testcase['Lastname']) else None
        self.driver.find_element(By.XPATH, self.xpath["AdminAddEmployeePage"]['CreateLogin']).click()

        time.sleep(self.TIMEOUT)
        self.driver.find_element(By.XPATH, self.xpath["AdminAddEmployeePage"]['StatusDisabled']).click()
        self.driver.find_element(By.XPATH, self.xpath["AdminAddEmployeePage"]['StatusEnabled']).click()
        self.driver.find_element(By.XPATH, self.xpath["AdminAddEmployeePage"]['Username']).send_keys(testcase["Username"]) if not pd.isna(testcase['Username']) else None
        self.driver.find_element(By.XPATH, self.xpath["AdminAddEmployeePage"]['Password']).send_keys(testcase["Password"]) if not pd.isna(testcase['Password']) else None
        self.driver.find_element(By.XPATH, self.xpath["AdminAddEmployeePage"]['ConfirmPassword']).send_keys(testcase["ConfirmPassword"]) if not pd.isna(testcase['ConfirmPassword']) else None

        time.sleep(self.TIMEOUT)
        saveButton.click()

        if testcase['Field'] == 'All':
            try:
                WebDriverWait(self.driver, self.TIMEOUT).until(
                    EC.presence_of_element_located(
                        (By.XPATH, self.xpath["AdminAddEmployeePage"]['SuccessNotification']))
                )
            except:
                print(f'Test case {testcase["No."]}: {"Success" if not testcase["ExpectedResult"] else "Fail"}')
                return
            print(f'Test case {testcase["No."]}: {"Success" if testcase["ExpectedResult"] else "Fail"}')
        else:
            try:
                WebDriverWait(self.driver, self.TIMEOUT).until(
                    EC.presence_of_element_located(
                        (By.XPATH, self.xpath["AdminAddEmployeePage"]['Invalid' + testcase['Field']]))
                )
            except:
                print(f'Test case {testcase["No."]}: {"Success" if testcase["ExpectedResult"] else "Fail"}')
                return
            print(f'Test case {testcase["No."]}: {"Success" if not testcase["ExpectedResult"] else "Fail"}')
    def RunAdminAddUser(self):
        print("AdminAddUser test:")
        for testcase in self.addUserTestCases:
            self.AdminAddUser(testcase)

    def RunAdminAddEmployee(self):
        for testcase in self.addEmployeeTestCases:
            self.AdminAddEmployee(testcase)
    
    def Quit(self):
        self.driver.quit()


In [ ]:
# Test: Chrome | AddUser
chromeDriver = OrangeHRMWebDriver(WebDriver.Chrome)
chromeDriver.AdminLogin()
chromeDriver.RunAdminAddUser()
chromeDriver.Quit()

In [ ]:
# Test: Chrome | AddEmployee
chromeDriver = OrangeHRMWebDriver(WebDriver.Chrome)
chromeDriver.AdminLogin()
chromeDriver.RunAdminAddEmployee()
chromeDriver.Quit()